# Matrix Decomposition

This notebook covers common matrix decomposition methods used in machine learning:
- QR Decomposition
- Eigen Decomposition
- Singular Value Decomposition (SVD)

## Setup

In [1]:
import numpy as np

## QR Decomposition

Decompose a matrix or set of vectors into an upper triangular matrix and an orthnormal basis using Gram-Schmidt.

$$A  = Q R$$

Where:
- $A \in \mathbb{R}^{n \times m}$
- $Q \in \mathbb{R}^{n \times m}$ is an orthonormal basis
- $R \in \mathbb{R}^{m \times m}$ is an upper triangular matrix of scalar projections

In [2]:
def qr_decompose(A):
    """
    Perform QR decomposition via Gram-Schmidt on a set of vectors

    Args:
        A: A numpy matrix n by m, where each column is a vector of length n

    Returns:
        Q: an n by m matrix that is an orthonormal basis
        R: an m by m upper triangular matrix of scalar projections
    """
    n, m = A.shape
    A = A.copy().astype(float)
    Q = np.zeros((n, m)).astype(float)
    R = np.zeros((m, m)).astype(float)

    # We can normalize beforehand to simplify projecting each vector onto every other vector
    # But that may lead to floating point errors, so let's prefer the standard method
    for j1 in range(m):
        vec_j1 = A[:, j1].copy()

        # Subtract projections of vec_j1 on previous vectors
        ortho_vec_j1 = A[:, j1].copy()
        for j2 in range(j1):
            vec_j2 = Q[:, j2].copy()

            # Normalize vec_j2
            mag_j2 = np.sqrt(np.sum(vec_j2 * vec_j2))
            norm_vec_j2 = vec_j2 / mag_j2

            # Get the scalar projection of vec_j1 onto vec_j2:
            # The scalar projection is typically vec_j1 @ vec_j2 / ||vec_j2||
            # But we have the normalized vector of vec_j2
            # And vec_j1 @ norm_vec_j2 = ||vec_j1|| * ||norm_vec_j2|| * cos(theta) = ||vec_j1|| * cos(theta)
            # Because vec_j1 is the hypotenus and cos(theta) = adj / hyp
            # Thus, ||vec_j1|| * cos(theta) = length of adj = length of vec_j1 projected onto norm_vec_j2
            # "A unit vector’s components are the cosines of its angles with the coordinate axes."
            scalar_proj_j1 = np.dot(vec_j1, norm_vec_j2)
            
            # Store the scalar projection of the original vector onto norm_vec_j2
            R[j2][j1] = scalar_proj_j1

            # Project vec_j1 onto vec_j2
            proj_j1_onto_j2 = scalar_proj_j1 * norm_vec_j2

            # Subtract the projection from ortho_vec_j1
            ortho_vec_j1 -= proj_j1_onto_j2

        # Normalize ortho_vec_j1
        mag_ortho_j1 = np.sqrt(np.sum(ortho_vec_j1 * ortho_vec_j1))
        orthonorm_vec_j1 = ortho_vec_j1 / mag_ortho_j1
        R[j1][j1] = mag_ortho_j1

        # Add orthonormal basis vector to Q
        Q[:, j1] = orthonorm_vec_j1

    return Q, R


sample_vectors = np.array([[1, 2], 
                           [0, 2]])
Q, R = qr_decompose(sample_vectors)
print('Q: \n', Q)
print('R: \n', R)
print('A: \n', Q @ R)

print('\n===\n')

sample_vectors = np.array([[1, 1, 0], 
                           [0, 1, 1], 
                           [0, 0, 1]])
Q, R = qr_decompose(sample_vectors)
print('Q: \n', Q)
print('R: \n', R)
print('A: \n', Q @ R)

print('\n===\n')

sample_vectors = np.array([[1, 1, 0], 
                           [1, 0, 1], 
                           [0, 1, 1]])
Q, R = qr_decompose(sample_vectors)
print('Q: \n', Q)
print('R: \n', R)
print('A: \n', np.round(Q @ R, decimals=10))

Q: 
 [[1. 0.]
 [0. 1.]]
R: 
 [[1. 2.]
 [0. 2.]]
A: 
 [[1. 2.]
 [0. 2.]]

===

Q: 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
R: 
 [[1. 1. 0.]
 [0. 1. 1.]
 [0. 0. 1.]]
A: 
 [[1. 1. 0.]
 [0. 1. 1.]
 [0. 0. 1.]]

===

Q: 
 [[ 0.70710678  0.40824829 -0.57735027]
 [ 0.70710678 -0.40824829  0.57735027]
 [ 0.          0.81649658  0.57735027]]
R: 
 [[1.41421356 0.70710678 0.70710678]
 [0.         1.22474487 0.40824829]
 [0.         0.         1.15470054]]
A: 
 [[ 1.  1. -0.]
 [ 1. -0.  1.]
 [ 0.  1.  1.]]


## Eigendecomposition

QR decomp takes a set of vectors and expresses them as combinations of orthonormal basis vectors and scalar projections. But eigendecompose operates on a linear transformation as a whole by expressing the matrix in terms of its eigenvalues and eigenvectors. In the basis of eigenvectors, the linear transformation scales each eigenvector by the eigenvalue so we see how the linear transformation modifies the space in the direction of each eigenvector.

$$A = V \Lambda V^{-1}$$

Where:
- $A \in \mathbb{R}^{n \times n}$ is a square matrix that is diagonalizable (has basis of eigenvectors)
- $V \in \mathbb{R}^{n \times n}$ is a matrix whose columns are the eigenvectors of $A$
- $\Lambda \in \mathbb{R}^{n \times n}$ is a diagonal matrix whose diagonal entries are the eigenvalues of $A$ corresponding to the eigenvectors in $V$
- $V^{-1}$ is the inverse of the eigenvector matrix $V$ (only exists if the eigenvectors are linearly independent)

In [ ]:
def eigendecompose(A):
    """
    Performs eigendecompose on a square matrix A

    Args:
        A: A numpy matrix n by n to decompose

    Returns:
        V: an n by n matrix of eigenvectors
        lambdas: diagonalized matrix of eigenvalues
        V_inverse: the inverse of V
    """
    n = A.shape[0]
    V = np.zeros((n, n)).astype(float)
    lambdas = np.zeros((n, n)).astype(float)
    V_inverse = np.zeros((n, n)).astype(float)

    # Getting eigenvalues by hand means getting the roots of the characteristic polynomial
    # for A - lambda * I. Eigenvectors of a linear transformation are vectors that are on 
    # the same span after applying the transformation and the eigenvalues are the multiplier 
    # that they scale by. Or mathematically, vectors such that A * v = lambda * v. For each 
    # eigenvalue, you want to find the null space of A - lambda * I that gives you the 
    # eigenvectors.
    eigenvals, eigenvecs = np.linalg.eig(A)

    for i in range(n):
        V[:, i] = eigenvecs[:, i]
        lambdas[i][i] = eigenvals[i]

    V_inverse = np.linalg.inv(V)

    return V, lambdas, V_inverse

sample_A = np.array([[1, 0, 0], 
                     [0, 1, 0], 
                     [0, 0, 1]])
V, lambdas, V_inverse = eigendecompose(sample_A)
print('V: \n', V)
print('lambdas: \n', lambdas)
print('V_inverse: \n', V_inverse)
print('V * lambdas * V_inverse: \n', V @ lambdas @ V_inverse)

print('\n===\n')

sample_A = np.array([[2, 0, 0], 
                     [0, 3, 0], 
                     [0, 0, 4]])
V, lambdas, V_inverse = eigendecompose(sample_A)
print('V: \n', V)
print('lambdas: \n', lambdas)
print('V_inverse: \n', V_inverse)
print('V * lambdas * V_inverse: \n', V @ lambdas @ V_inverse)

print('\n===\n')

sample_A = np.array([[2, 1, 0], 
                     [1, 2, 1], 
                     [0, 1, 2]])
V, lambdas, V_inverse = eigendecompose(sample_A)
print('V: \n', V)
print('lambdas: \n', lambdas)
print('V_inverse: \n', V_inverse)
print('V * lambdas * V_inverse: \n', np.round(V @ lambdas @ V_inverse, decimals=10))

V: 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
lambdas: 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
V_inverse: 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
V * lambdas * V_inverse: 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

===

V: 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
lambdas: 
 [[2. 0. 0.]
 [0. 3. 0.]
 [0. 0. 4.]]
V_inverse: 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
V * lambdas * V_inverse: 
 [[2. 0. 0.]
 [0. 3. 0.]
 [0. 0. 4.]]

===

V: 
 [[-5.00000000e-01  7.07106781e-01  5.00000000e-01]
 [-7.07106781e-01  4.05405432e-16 -7.07106781e-01]
 [-5.00000000e-01 -7.07106781e-01  5.00000000e-01]]
lambdas: 
 [[3.41421356 0.         0.        ]
 [0.         2.         0.        ]
 [0.         0.         0.58578644]]
V_inverse: 
 [[-5.00000000e-01 -7.07106781e-01 -5.00000000e-01]
 [ 7.07106781e-01  3.14018492e-16 -7.07106781e-01]
 [ 5.00000000e-01 -7.07106781e-01  5.00000000e-01]]
V * lambdas * V_inverse: 
 [[ 2.  1. -0.]
 [ 1.  2.  1.]
 [-0.  1.  2.]]


# Singular Value Decomposition

So SVD is a generalized form of eigendecompose that applies to any linear transformation that maps vectors from an input space of rank $m$ to an output space of rank $n$. It decomposes a transformation into the rotation in the input space (domain basis), scaling along new axes (singlular values), and rotation into the output space (range basis). The computation itself doesn't seem to have a nice geometric intuition compared to the final components of SVD. But the algebraic derivation from the formula involves transposes of orthogonal bases and that $A * v_i = \epsilon_i * u_i$ where $v_i$ is in the input space and $u_i$ is in the output space, which is oddly similar to another concept.

$$A = U \Sigma V^T$$

Where:
- $A \in \mathbb{R}^{m \times n}$ is any real matrix (square or rectangular)
- $U \in \mathbb{R}^{m \times m}$ is an orthogonal matrix whose columns are the left singular vectors of $A$
- $\Sigma \in \mathbb{R}^{m \times n}$ is a diagonal matrix (with non-negative real numbers) containing the singular values of $A$ on the diagonal, sorted in decreasing order
- $V \in \mathbb{R}^{n \times n}$ is an orthogonal matrix whose columns are the right singular vectors of $A$
- $V^T$ is the transpose of the right singular vector matrix

In [ ]:
def svd(A):
    """
    Performs singular value decomposition on an m x n matrix A

    Args:
        A: A numpy matrix m by n to decompose

    Returns:
        U: an m by m orthogonal matrix with left singular values
        E: an m by n diagonal matrix with non-negative singular values
        V: an n by n orthogonal matrix with right singular values
        where A = U @ E @ V^T
    """
    m, n = A.shape
    U = np.zeros((m, m))
    E = np.zeros((m, n))
    V = np.zeros((n, n))

    # A^T @ A = V @ E @ V^T
    # We need to eigendecompose the gram matrix.
    gram = np.transpose(A) @ A
    eigenvals, eigenvecs = np.linalg.eigh(gram)

    # We want to sort by decreasing eigenvalue to make it easier afterwards when we
    # want to generate U, we can leave all the eigenvalues of 0 at the end
    indices = np.argsort(eigenvals)[::-1]
    eigenvals = eigenvals[indices]
    eigenvecs = eigenvecs[:, indices]
    for i in range(n):
        # Eigenvectors are already normalized
        V[:, i] = eigenvecs[:, i]

    # Singular vals are sqrt(eigenvals of A^T @ A)
    singular_vals = np.sqrt(eigenvals)
    for i in range(min(m, n)):
        E[i, i] = singular_vals[i]

    # Generate u_i = (1 / singular_val_i) * (A @ v_i)
    r = min(len([singular_val for singular_val in singular_vals if singular_val > 1e-10]), m)
    for i in range(r):
        v_i = V[:, i]
        u_i = (1 / singular_vals[i]) * (A @ v_i)
        U[:, i] = u_i

    # Fill in the rest of U with any orthonormal basis
    Q = U[:, :r]
    candidates = np.random.randn(m, m - r)
    for i in range(r):
        q_i = Q[:, i]
        for j in range(m - r):
            # q_i is already a unit vec so the dot product should be the scalar projection
            proj = np.dot(q_i, candidates[:, j])

            # We can subtract the projection from the candidates
            candidates[:, j] -= proj * q_i
        
    # Orthonormalize the resulting columns
    Q_new, _ = qr_decompose(candidates)
    U[:, r:] = Q_new

    return U, E, V

sample_A = np.array([[3, 1],
                     [2, 2],
                     [1, 3]], dtype=float)

U, E, V = svd(sample_A)

print('U:\n', U)
print('E:\n', E)
print('V:\n', V)
print('U @ E @ V.T:\n', np.round(U @ E @ V.T, decimals=10))

U:
 [[ 0.57735027 -0.70710678 -0.40824829]
 [ 0.57735027  0.          0.81649658]
 [ 0.57735027  0.70710678 -0.40824829]]
E:
 [[4.89897949 0.        ]
 [0.         2.        ]
 [0.         0.        ]]
V:
 [[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]]
U @ E @ V.T:
 [[3. 1.]
 [2. 2.]
 [1. 3.]]
